In [1]:
# Устанавливаем необходимые пакеты
!pip install justwatch-python3 requests beautifulsoup4 pandas tqdm unidecode
# justwatch-python3 — форк/обертка, если не сработает, попробуй justwatch (pip install justwatch)


ERROR: Could not find a version that satisfies the requirement justwatch-python3 (from versions: none)
ERROR: No matching distribution found for justwatch-python3


In [1]:
!pip install justwatch

  Preparing metadata (setup.py) ... done
  Created wheel for justwatch: filename=JustWatch-0.5.1-py3-none-any.whl size=3736 sha256=b9f88ca589d5721b71273ac82131e699ef0bd62a78dd24e8f4edab7d47a2181f
  Stored in directory: /root/.cache/pip/wheels/72/35/51/1458d5fa4f222352a8442c5cf5d4bcec2dc057626036cd1788
Successfully built justwatch


In [3]:

!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 8.2 MB/s eta 0:00:00


In [4]:
import time
import random
import requests
from bs4 import BeautifulSoup
from justwatch import JustWatch
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode
import os

# Настройки
TARGET = 2000        # сколько записей в итоге хотим собрать
OUTPUT_CSV = "tvshows_dataset.csv"
COUNTRY = 'RU'       # Лучше использовать RU для русских описаний, если доступны.
DELAY_MIN = 0.8      # минимальная задержка между внешними запросами (сек)
DELAY_MAX = 1.8      # максимальная задержка

In [5]:
genre_queries = [
    "dramatic reality", "human comedy", "magical realism", "action", "urban romance", "noir",
    "tragicomedy", "paranormal", "arthouse", "horror comedy", "dark fantasy", "comic detective",
    "parody", "retro drama", "black comedy", "alternate history"
]


In [6]:
# Инициализируем JustWatch
justwatch = JustWatch(country=COUNTRY)

def search_justwatch_by_query(query, page=1, page_size=50, content_type='show'):
    """
    Выполнит поиск по JustWatch и вернёт список результатов (словарей)
    content_type: 'show' или 'movie'
    """
    try:
        res = justwatch.search_for_item(
            query=query,
            page=page,
            page_size=page_size,
            content_types=[content_type]
        )
        items = res.get('items', [])
        return items
    except Exception as e:
        print("JustWatch search error:", e)
        return []

def get_jw_item_info(item):
    """
    Берёт item из justwatch и возвращает поля: title, description, poster, jw_url
    """
    title = item.get('title') or item.get('object', {}).get('title')
    # description: justwatch may have 'short_description' or 'full_path' etc.
    description = item.get('short_description') or item.get('short_description') or item.get('overview') or ''
    poster = None
    images = item.get('poster', None) or item.get('images', None)
    if isinstance(images, dict):
        # JustWatch stores poster resources as url templates sometimes
        poster = images.get('standard') or images.get('poster')
    elif isinstance(images, str):
        poster = images
    # build JustWatch page URL (approx)
    jw_url = item.get('offers', [])
    # If item has 'object': build URL using 'id' if possible
    item_id = item.get('id') or item.get('object', {}).get('id')
    jw_page = None
    if item_id:
        jw_page = f"https://www.justwatch.com/{COUNTRY.lower()}/tv/{item_id}"
    return {
        "title": title or '',
        "description": description or '',
        "poster": poster,
        "jw_page": jw_page
    }

def search_imdb_for_title(title):
    """
    Поиск IMDb: возвращает первый результат URL и краткое описание, постер при возможности.
    Небольшой парсер страницы поиска.
    """
    try:
        q = requests.utils.requote_uri(title)
        url = f"https://www.imdb.com/find?q={q}&s=tt&ttype=tv&ref_=fn_tv"
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        # Найдём первую строку результата
        first = soup.select_one(".findList .findResult a")
        if first and first.has_attr("href"):
            href = first['href']
            # полноый imdb url
            imdb_url = requests.compat.urljoin("https://www.imdb.com/", href.split('?')[0])
            # зайдём на imdb_url и попробуем достать описание и постер
            time.sleep(random.uniform(DELAY_MIN, DELAY_MAX))
            r2 = requests.get(imdb_url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            r2.raise_for_status()
            soup2 = BeautifulSoup(r2.text, "html.parser")
            desc_tag = soup2.select_one("div[data-testid='plot-xl']") or soup2.select_one(".summary_text")
            desc = desc_tag.get_text(strip=True) if desc_tag else ''
            poster_tag = soup2.select_one(".ipc-media img") or soup2.select_one(".poster img")
            poster = poster_tag['src'] if poster_tag and poster_tag.has_attr('src') else None
            return imdb_url, desc, poster
        else:
            return None, None, None
    except Exception as e:
        # print("IMDB search error:", e)
        return None, None, None

In [11]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

TARGET = 2000
OUTPUT_CSV = "tvshows_tvmaze.csv"

def fetch_shows_page(page):
    url = f"https://api.tvmaze.com/shows?page={page}"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    return r.json()

def fetch_show_details(show_id):
    url = f"https://api.tvmaze.com/shows/{show_id}?embed=cast"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    return r.json()

collected = []
seen_ids = set()
page = 0

# Перечень жанров английских, которые тебе нужны
target_genres_en = [
    "Dramatic reality", "Human comedy", "Magical realism", "Action", "Urban romance", "Noir",
    "Tragicomedy", "Paranormal", "Arthouse", "Horror comedy", "Dark fantasy",
    "Comic detective", "Parody", "Retro drama", "Black comedy", "Alternate history"
]

pbar = tqdm(total=TARGET, desc="Collecting TVMaze shows")
while len(collected) < TARGET:
    shows = fetch_shows_page(page)
    if not shows:
        break
    for s in shows:
        if len(collected) >= TARGET:
            break
        sid = s.get('id')
        if sid in seen_ids:
            continue
        seen_ids.add(sid)
        genres = s.get('genres', [])
        # фильтр: хотя бы один из жанров совпадает с target
        if not any(g.lower() in [tg.lower() for tg in target_genres_en] for g in genres):
            continue
        # получаем детали
        try:
            details = fetch_show_details(sid)
        except:
            continue
        title = details.get('name')
        description = details.get('summary') or ''
        # удалим html-тэги
        description = pd.Series(description).replace({r'<.*?>':''}, regex=True).iloc[0]
        year = (details.get('premiered') or '')[:4]
        image_url = details.get('image', {}).get('original') if details.get('image') else ''
        rating = details.get('rating', {}).get('average')
        cast_info = details.get('_embedded', {}).get('cast', [])
        actors = ", ".join([c.get('person',{}).get('name','') for c in cast_info[:5]])
        director = ""  # TVMaze API может не давать режиссёра напрямую
        page_url = details.get('url')
        collected.append({
            "page_url": page_url,
            "image_url": image_url,
            "tvshow_title": title,
            "description": description,
            "genre": "; ".join(genres),
            "year": year,
            "director": director,
            "actors": actors,
            "rating": rating
        })
        pbar.update(1)
        if len(collected) >= TARGET:
            break
        time.sleep(random.uniform(0.5, 1.5))
    page += 1

pbar.close()
df = pd.DataFrame(collected)
df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
print("✅ Saved", len(df), "shows to", OUTPUT_CSV)
df.head()


✅ Saved 2000 shows to tvshows_tvmaze.csv


,page_url,image_url,tvshow_title,description,genre,year,director,actors,rating
0,https://www.tvmaze.com/shows/2/person-of-interest,https://static.tvmaze.com/uploads/images/origi...,Person of Interest,You are being watched. The government has a se...,Action; Crime; Science-Fiction,2011,,"Jim Caviezel, Michael Emerson, Kevin Chapman, ...",8.8
1,https://www.tvmaze.com/shows/4/arrow,https://static.tvmaze.com/uploads/images/origi...,Arrow,"After a violent shipwreck, billionaire playboy...",Drama; Action; Science-Fiction,2012,,"Stephen Amell, David Ramsey, Emily Bett Rickar...",7.4
2,https://www.tvmaze.com/shows/6/the-100,https://static.tvmaze.com/uploads/images/origi...,The 100,"Ninety-seven years ago, nuclear Armageddon dec...",Action; Adventure; Science-Fiction,2014,,"Eliza Taylor, Marie Avgeropoulos, Bob Morley, ...",7.7
3,https://www.tvmaze.com/shows/11/gotham,https://static.tvmaze.com/uploads/images/origi...,Gotham,The good. The evil. The beginning.Everyone kno...,Drama; Action; Crime,2014,,"Ben McKenzie, Donal Logue, David Mazouz, Robin...",7.7
4,https://www.tvmaze.com/shows/13/the-flash,https://static.tvmaze.com/uploads/images/origi...,The Flash,After a particle accelerator causes a freak st...,Drama; Action; Science-Fiction,2014,,"Grant Gustin, Candice Patton, Danielle Panabak...",7.5


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
# путь, куда сохранить
save_path = '/content/drive/MyDrive/tvshows_tvmaze.csv'

# сохраняем
df.to_csv(save_path, index=False, encoding='utf-8')
print(f"✅ Файл сохранён на Google Диск: {save_path}")

✅ Файл сохранён на Google Диск: /content/drive/MyDrive/tvshows_tvmaze.csv
